# Technical Indicators

## SMA

Simple Moving Average (SMA) is a technical indicator that is commonly used in financial analysis and trading. Moving averages are versatile tools that can be used to identify trends, support and resistance levels, and other trading opportunities.

Simple moving averages can be used in various ways, including:

- Trend Identification: Moving averages can help identify the overall trend of a security by smoothing out fluctuations in the price.
- Support and Resistance: Moving averages can act as support or resistance levels, as they provide a visual representation of the average price of a security over a given time period.
- Buy and Sell Signals: Moving averages can be used to generate buy and sell signals by combining them with other technical indicators or chart patterns.
- Momentum Indicator: Moving averages can be used as a momentum indicator, which measures the rate of change in the price of a security.

## EMA

An Exponential Moving Average (EMA) is a technical indicator that is often used in financial analysis and trading. Its use cases and calculations are very similar to Simple Moving Averages (SMA) but with an added variable of weights.

Unlike Simple Moving Averages which give equal weight to every period, Exponential Moving Averages give higher weights to the most recent periods, as to better reflect the latest price action.

Exponential moving averages can be used in various ways, including:

- Trend Identification: Moving averages can help identify the overall trend of a security by smoothing out fluctuations in the price.
- Support and Resistance: Moving averages can act as support or resistance levels, as they provide a visual representation of the average price of a security over a given time period.
- Buy and Sell Signals: Moving averages can be used to generate buy and sell signals by combining them with other technical indicators or chart patterns.
- Momentum Indicator: Moving averages can be used as a momentum indicator, which measures the rate of change in the price of a security.


# Trading Strategy

## Chose Underlying Asset

- Select Highly liquid stocks with a history of significant price movements.This liquidity is crucial for finding options with tighter bid-ask spreads.

## Technical Indicators

- Moving Averages use the crossover of a short-term MA (10 days) above a long term MA (50 days) as bullish signal. Conversely, a crossover below signals bearish conditions.
- Relative strength index (RSI): An RSI below 30 indicates oversold conditions (potential for bullish reversal), while 70 suggests overbought conditions (potential for bearish reversal)
- Support and Resistance Levels: Identify key price levels. Buying calls when the price bounces of a support or buying puts when it falls from resistance can be effective.

## Strategy for Buying Calls

- Bullish Signal: Look for a bullish crossover in MAs, RSI moving above 30 (indicating a move from oversold to normal), and a bounce off a support level.
- Timing: Buy calls 20-30 days to expiration to balance between time decay and the cost of the option. Opt for slightly out-of-the-money (OTM) calls to potentially increase returns.
- Exit Plan: Set a profit target based on resistance levels or a percentage gain. Implement a stop-loss to limit losses if the trade moves against you.

## Strategy for Buying Puts

- Bearish Signal: A bearish MA crossover, RSI moving below 70 (indicating a move from overbought to normal), and a rejection at a resistance level suggest a bearish turn.
- Timing: Purchase puts 20-30 days to expiration. Choose slightly OTM puts to manage costs while retaining the potential for profit.
- Exit Plan: Define a profit target based on support levels or a set percentage gain. Use a stop-loss to protect against significant losses.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import yfinance as yf
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Back testing
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

In [ ]:
API_KEY = 'ec2d6f3b0cc9395fbce02f1316669229'
ticker = 'CCJ'

In [ ]:
#Earning calendar 
url = f'https://financialmodelingprep.com/api/v3/historical/earning_calendar/{ticker}L&apikey={API_KEY}'
calendar = pd.read_json(url)
print(calendar)

In [ ]:
url = f'https://financialmodelingprep.com/api/v3/technical_indicator/1day/{ticker}?type=rsi&period=10&apikey={API_KEY}'
data = pd.read_json(url)
RSI = pd.DataFrame(data)

In [ ]:
url = f'https://financialmodelingprep.com/api/v3/technical_indicator/1day/{ticker}?type=sma&period=10&apikey={API_KEY}'
data = pd.read_json(url)
SMA10 = pd.DataFrame(data)

In [ ]:
url = f'https://financialmodelingprep.com/api/v3/technical_indicator/1day/{ticker}?type=sma&period=50&apikey={API_KEY}'
data = pd.read_json(url)
SMA50 = pd.DataFrame(data)

In [ ]:
for x in [RSI, SMA10]: 
    x.drop(['open', 'high', 'low', 'volume', 'date'], axis=1, inplace=True)

In [ ]:
#Renam SMA Columns to their periods 
SMA10.rename(columns={'sma':'SMA10'}, inplace=True)
SMA50.rename(columns={'sma':'SMA50'}, inplace=True)

In [ ]:


data = pd.concat([SMA50, SMA10, RSI], axis='columns', join='inner')
data = data.sort_values(by='date')

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

In [ ]:
#drop duplicate columns
data = data.loc[:,~data.columns.duplicated()]

In [ ]:
data

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(data['SMA10'], label='SMA10', color='orange')
plt.plot(data['SMA50'], label='SMA50', color='blue')
plt.plot(data['close'], label='Close', color='green')
plt.title('SMA10 vs SMA50')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend(loc='upper left')
plt.show()

In [ ]:
#pct_change
data['daily_return']= data['close'].pct_change(-1) * 100 * -1

In [ ]:
data['change_direction'] = np.where(data['daily_return'] > 0, 'UP', 'DOWN')

In [ ]:
data = data.dropna()

In [ ]:
data['change_direction'].value_counts()

In [ ]:
#capitalize the first letter of each column
data.columns = data.columns.str.capitalize()

In [ ]:
data


## Machine Learning Model

In [ ]:
y = data.Change_direction
x = data[['Close', 'High', 'Low', 'Volume']]

In [ ]:
#make uppercase just the first letter of each column
x.columns = x.columns.str.capitalize()

In [ ]:
#split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
model.fit(X=x_train, y=y_train)

In [ ]:
y_pred = model.predict(X=x_test) 

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
#comparison data frame
comparison = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
comparison

In [ ]:
comp  = comparison['Actual'] == comparison['Predicted']

In [ ]:
comp.sum()

In [ ]:
len(comp)

In [ ]:
comp.sum() / len(comp)

## Backtesting

In [ ]:

x_today = x.iloc[[-1], :]

In [ ]:
x_today

In [ ]:
forecast = model.predict(x_today)[0]

In [ ]:
forecast

In [ ]:
data.sort_index(inplace=True)

In [ ]:
data

In [ ]:
class MAOptionStrategy(Strategy):
    def init(self):
        self.sma10 = self.I(lambda x: x, self.data.Sma10)
        self.sma50 = self.I(lambda x: x, self.data.Sma50)

    def next(self):
        # Entry: Buy when Sma10 crosses above Sma50 if not already in position
        if not self.position:
            if crossover(self.sma10, self.sma50):
                self.buy()
        # Exit: Sell when Sma50 crosses above Sma10 or add your specific exit criteria
        else:
            if crossover(self.sma50, self.sma10):
                self.position.close()

# Assuming 'data' is your DataFrame properly formatted and indexed by DateTime
bt = Backtest(data, MAOptionStrategy, cash=10_000, commission=.002)
stats = bt.run()
print(stats)

In [ ]:
bt.plot()

## New Trading Strategy


In [ ]:
from pandas_datareader import data
import yfinance as yf 
import quandl  

In [ ]:
ticker = 'CCJ'
df = yf.download(ticker, start='2000-01-01', end='2024-04-11')

In [ ]:
df.head()

In [ ]:
df1  =  quandl.get("WIKI/AAPL", start='2000-01-01', end='2024-04-11')

In [ ]:
df1

# Options
Pay off from a call option  
K = Strike price 
St = final price of the underlying asset 
max(St-K,0)
if St>K --> Option will be excercised 
and if St <= K will not be excercised

pay off from a short position call option 
-max(St-K,0) = min(K-St,0) 

The payoff to the holder of a long position  put option 
max(K-St,0) 

The pay off from  a shor put is 
-max(K-St,0) = min(St-K,0)

## Call Option 
S = Stock Price
K = Strike Price 
if S > K --> in the money 

if S = K --> At the money

if S < K --> Out of the Money 

## Put Option 

if S < K --> in the money 

if S = K --> at the money

if S > K --> out of the money 

## Intrinsic Value 
Is define as the value it would have if there were no time to maturity, so that the excercise decision has to be made immediatley 
For a call option the intrinsic value  = max(S-K,0) 
For a put max(K-S,0) 

The excess of an options value over its intrinsic value is the options time value. The total value of an option is therefore the sum of its intrinsic value and its time value 


# Factors Affecting Option Prices

S = Current Stock Price
K = Strike Price
T = Time to Expiration
ò = Volatility of the stock price
r = Risk free rate of return


S0 = Current Stock Price
K = Strike Price
T = Time to Expiration
St = Stock price at time T(Expiration)
r = continuously compounded risk-free rate of interest for an investment maturing at time T(Expiration)
C = value of an American call option
P = value of an American put option
c = value of a European call option
p = value of a European put option


## Upper Bounds 
C <= S0
c <= S0
If this were not the case, it would be possible to make a riskless profit by buying the stock and selling the call option.

P <= K (American)
p <= ke^(-rT) (European)
If this were not the case, it would be possible to make a riskless profit by buying the option and investing the proceeds at the risk-free rate.

## Lower Bounds
for european put option a no dividend paying stock, a lower bound for the price is 
Ke^(-rT) - S0

In [3]:
import pandas as pd
import requests
import base64


Click to authenticate: https://api.schwabapi.com/v1/oauth/authorize?client_id=&redirect_uri=https://127.0.0.1
